# 🐅 Парсер

Парсинг начинается со страниц, перечисленных в `pages.json`. Файл содержит необходимые сведения о структуре страниц сайта: стартовая страница, селекторы страницы, отвечающие за их описания (могут быть на стартовой `start_url` или внутренних страницах `event_urk`), количество соответствующих элементов. Если для ссылки события пока нет описания в файле `events.json`, в результате парсинга создаётся соответствующий объект. Этот файл используется для генерации карточек мероприятий в HTML, публикуемый на [matyushkin.github.io/events](https://matyushkin.github.io/events/).

In [1]:
import json

# custom packages
import files
import soups
import urls
import langs
import handlers

# Узнаем, какие из стартовых страниц, перечисленных в files.pages, размечены.
# Такие страницы содержат все поля, перечисленные в files.fields.
# Выводим список с помощью специальной функции pages_checked()

pages_checked = files.pages_checked()

# Для обработки полей, описывающих события, логична следующая последовательность:
# - cчитываемые поля со стартовой страницы (обычно это заголовк, дата, статус)
# - поля, расположенные на странице мероприятия (темы, докладчики, время начала)
# - получаемые из анализа предобработанных полей (теги, призы)


class StartPage:
    """Парсинг начальных страниц, описанных в pages.json"""
    def __init__(self, start_url):
        self.data, self.event_lists, self.events = {}, {}, {}
        self.data['start_url'] = start_url
        self.fields = files.fields_order(start_url)
        for field in self.fields['start']:
            self.event_lists[field] = files.get_content(self.data, field, force=True)
        event_urls = self.event_lists.pop('event_url')
        self.fields['start'].remove('event_url')
        for i, event_url in enumerate(event_urls):
            self.events[event_url] = {}
            for field in self.fields['start']:
                self.events[event_url][field] = self.event_lists[field][i]
    

    def __repr__(self):
        return self.actual_events
            
                
    @property
    def actual_events(self):
        """События, актуальные на текущий день"""
        events = self.events.copy()
        for event_url in self.events:
            if events[event_url]['date'] < handlers.current_date.isoformat():
                events.pop(event_url)
        return events


class EventPage:
    """Парсинг страниц событий"""
    def __init__(self, event_url, start_page, force=False):
        self.data = start_page.events[event_url]
        self.data['start_url'] = start_page.data['start_url']
        self.data['event_url'] = event_url
        for field in start_page.fields['event']:
            self.data[field] = files.get_content(self.data, field, force)
            


for start_url in pages_checked:
    start_page = StartPage(start_url)
    actual = start_page.actual_events
    print(actual)
    for event_url in actual:
        event = EventPage(event_url, start_page)
        files.events[event_url] = event.data  #! Спорное решение, требует замены

{'https://events.yandex.ru/events/mini-giperbaton-22-07-2020': {'date': '2020-07-22', 'title': 'Мини-Гипербатон. Онлайн'}, 'https://events.yandex.ru/events/yasubbotnik/4-jul-2020': {'date': '2020-07-04', 'title': 'Я.Субботник по разработке интерфейсов. Онлайн, темы выбраны!'}, 'https://cloud.yandex.ru/events/145': {'date': '2020-07-02', 'title': 'about:cloud – бессерверные технологии и IoT'}, 'https://cloud.yandex.ru/events/143': {'date': '2020-06-30', 'title': 'Вебинар. Как работает сеть в Облаке'}}
module 'handlers' has no attribute 'themes'
module 'handlers' has no attribute 'speakers'
module 'handlers' has no attribute 'organizers'
module 'handlers' has no attribute 'speakers_companies'
module 'handlers' has no attribute 'description'
module 'handlers' has no attribute 'price'
module 'handlers' has no attribute 'location'
module 'handlers' has no attribute 'themes'
module 'handlers' has no attribute 'speakers'
module 'handlers' has no attribute 'organizers'
module 'handlers' has no

In [3]:
files.events

{'https://events.yandex.ru/events/yasubbotnik/4-jul-2020': {'date': '2020-07-04',
  'title': 'Я.Субботник по разработке интерфейсов. Онлайн, темы выбраны!',
  'start_url': 'https://events.yandex.ru/',
  'event_url': 'https://events.yandex.ru/events/yasubbotnik/4-jul-2020',
  'themes': ['Бесконечная лента — без шуток',
   'Как померить Node.js приложения, если у тебя лапки',
   '@yandex/ui',
   'Эмуляция NES/Famicom/Dendy на веб-технологиях',
   'Магия современных возможностей JavaScript для работающего программиста',
   'Общение',
   'Бесконечная лента — без шуток',
   'Как померить Node.js приложения, если у тебя лапки',
   '@yandex/ui',
   'Эмуляция NES/Famicom/Dendy на веб-технологиях',
   'Магия современных возможностей JavaScript для работающего программиста',
   'Общение'],
  'online_status': 'Online',
  'speakers': ['Валентин Каменек',
   'Алексей Попков',
   'Владимир Гриненко',
   'Александр Николаичев',
   'Евгений Кузнецов'],
  'organizers': ['Яндекс'],
  'speakers_companies

In [ ]:
with open('files/events.json', 'w', encoding='utf-8') as events_file:
    json.dump(files.events, events_file, ensure_ascii=False)

In [ ]:
#! Отсюда можно забрать обработчикитегов

# all_fields = set(fields.keys())
# required   = {key for key in fields if fields[key]['required']}
# optional   = {key for key in fields if not fields[key]['required']}

#     def tags(self):
#         tags = {}
#         title = self.data['title']
#         event_url = self.data['event_url']
#         themes = self.data['themes'].copy()
#         themes.append(title)
#         themes.append(event_url)
#         text = ''.join(themes)
#         for key in tags:
#             for tag in tags[key]:
#                 if tag.lower() in text.lower():
#                     self.data[tags].add(tag)
#         return tags

# Собираем HTML-страницу и деплоим проект 💃

In [ ]:
from bs4 import BeautifulSoup, NavigableString
import pandas as pd

soup = BeautifulSoup('<article></article')
    
df = pd.DataFrame.from_dict(files.events, orient='index')
df = df.sort_values(by=['date'])
df = df[df['date'] >= handlers.current_date.isoformat()]   #! добавить проверку на время


def add_tag(child, parent, text='', attrs={}):
    tag = soup.new_tag(child, attrs=attrs)
    tag.string = text
    parent.append(tag)
    return eval(f'parent.{child}')


for event_url in df.index:
    data = df.loc[event_url]
    attrs = {'data-date': data.date, 'data-time': data.time, 'data-organizers': data['organizers']}
    section = soup.new_tag('section', attrs)
    hr = soup.new_tag('hr')
    section.append(hr)

    h2 = add_tag('h2', section, data.title)
    p = add_tag('p', section)
    datetime_str = langs.make_datetime_string(data.date, data.time)
    t = add_tag('time', p, datetime_str)
    p.append(NavigableString(", "))
    add_tag('a', p, text='ссылка на мероприятие', attrs={'href': event_url})
    
    if data.reg_url:
        p.append(NavigableString(", "))
        add_tag('a', p, text='ссылка для регистрации', attrs={'href': data.reg_url})

    if data.themes != ['']:
        themes_ul = add_tag('ul', section, attrs={'class': 'theme-block'})

        for theme in set(data.themes):
            theme_li = add_tag('li', themes_ul, theme, {'class': 'theme'})

    description = add_tag('p', section, data.description[0], attrs={'class':'description__text'})
    soup.article.append(section)

hr = soup.new_tag('hr')
soup.article.append(hr)

path_to_html_template = "../mgio/11ty/_includes/events.njk"
with open(path_to_html_template, 'w', encoding='utf-8') as html_template:
    html_template.write(str(soup.article))

In [ ]:
!cd ../mgio/11ty/; npx eleventy --passthroughall --output=../../matyushkin.github.io
!cd ../matyushkin.github.io/; rm -rf 404
!cd ../matyushkin.github.io/; rm -rf README


# посмотрим, что получилось в браузере
import webbrowser
url = "../matyushkin.github.io/events/index.html"
webbrowser.open_new_tab(url)

In [ ]:
! cd ../matyushkin.github.io/; git add . ; git commit -m "Events page is updated: only actual events"; git push origin master